In [1]:
# import libraries
import pandas as pd
import numpy as np
import sqlite3


import re
import nltk
nltk.download('punkt')
nltk.download('wordnet')
nltk.download('stopwords')

from nltk.tokenize import word_tokenize
from nltk.tokenize import sent_tokenize
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords


from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report


from sklearn.ensemble import AdaBoostClassifier
from sklearn.tree import DecisionTreeClassifier

from sklearn.model_selection import GridSearchCV
from sklearn.neural_network import MLPClassifier


[nltk_data] Downloading package punkt to /Users/vikram/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/vikram/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/vikram/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [22]:
# Extracting 'elog_data_2011' table from 'elog_data.db' database
conn = sqlite3.connect(r'/Users/vikram/Projects/Elog/data/elog_data.db')
c = conn.cursor()
c.execute('SELECT * FROM elog_data_2011')
df = pd.DataFrame(c.fetchall(), columns=['elogid', 'tag', 'text', 'title', 'title_and_text'])
df.shape

(6913, 5)

In [19]:
df.tail(10)

elogid   tag                                               text  \
251183  968143  LCLS  I'm confident that the LTU orbit is okay on al...   
251184  968142  LCLS  +38/-76 MeV at BC2, +7/-3.5 MeV at DL2. XTCAV ...   
251185  968140  LCLS                                                      
251186  968138  None  LCLS: CXI delivery, XPP/MFX delivery.\nLCLS-II...   
251187  968136  None  <table CellPadding="5" BORDER=1>\n<tr>\n<th>Co...   
251188  968135  LCLS                                                      
251189  968133  LCLS                                                      
251190  968132  LCLS                                                      
251191  968130  None  LCLS BCS checks performed using the 10/30/2018...   
251192  968131  None  LCLS: Deliver to CXI at 9.5keV all shift. \nES...   

                                                    title  \
251183  Guardian keeps tripping on LTU orbit, increasi...   
251184        Restored previous data slot offsets for XPP   
251185                 CXI finished, switching to XPP/MFX   
251186  MCC Shift Change: Day Shift, Saturday, 01-Dec-...   
251187                                  OWL SHIFT SUMMARY   
251188     NEH/FEH walkthrough complete--No Trouble Found   
251189     NEH/FEH walkthrough complete--No Trouble Found   
251190     NEH/FEH walkthrough complete--No Trouble Found   
251191              Daily BCS Checks - 12/1/2018 - Passed   
251192  MCC Shift Change: Owl Shift, Saturday, 01-Dec-...   

                                           title_and_text  
251183  Guardian keeps tripping on LTU orbit, increasi...  
251184  Restored previous data slot offsets for XPP +3...  
251185                CXI finished, switching to XPP/MFX   
251186  MCC Shift Change: Day Shift, Saturday, 01-Dec-...  
251187  OWL SHIFT SUMMARY <table CellPadding="5" BORDE...  
251188    NEH/FEH walkthrough complete--No Trouble Found   
251189    NEH/FEH walkthrough complete--No Trouble Found   
251190    NEH/FEH walkthrough complete--No Trouble Found   
251191  Daily BCS Checks - 12/1/2018 - Passed LCLS BCS...  
251192  MCC Shift Change: Owl Shift, Saturday, 01-Dec-...

> Questions to be answere later: Does it perform better all lower or all uppercase??

In [83]:
# Turning it all to lower case and removing newline stuff
df['title_and_text'] = df['title_and_text'].str.lower()
df['title_and_text'] = df['title_and_text'].str.replace('\n', ' ')
df.tail(10)

elogid   tag                                               text  \
6903  554308  LCLS  Put in Laser heater for diagnostics and switch...   
6904  554298  LCLS  Garth is hoping that Jim Turner identified the...   
6905  554292  LCLS  Jim Craft has left a note on CR 2 of LI21 that...   
6906  554290  LCLS                                                      
6907  554265  LCLS                                                      
6908  554263  LCLS                                                      
6909  554261  LCLS                                                      
6910  554258  LCLS  Will LEM the change in; seems to be alright so...   
6911  554257  LCLS                                                      
6912  554250  LCLS  <table class=emittanceTable>\n<tr><th></th><th...   

                                                  title  \
6903  Laser heater motor delay has lost "home" position   
6904  Garth at CXI is seeing lower Intensity than GD...   
6905                     Jim Craft advisory on CR2 LI21   
6906                  repointUndulatorLine(0,0,-45,-45)   
6907  Switching to Matlab UND launch. Fast Und launc...   
6908                Model manager hangs on "gold" stage   
6909                               BBA Round 2 complete   
6910                              Swapped 30-6 for 30-4   
6911                                 1st Round BBA done   
6912  Emittance Measurements - 9491  eV (14.673  GeV...   

                                         title_and_text  
6903  laser heater motor delay has lost "home" posit...  
6904  garth at cxi is seeing lower intensity than gd...  
6905  jim craft advisory on cr2 li21 jim craft has l...  
6906                 repointundulatorline(0,0,-45,-45)   
6907  switching to matlab und launch. fast und launc...  
6908               model manager hangs on "gold" stage   
6909                              bba round 2 complete   
6910  swapped 30-6 for 30-4 will lem the change in; ...  
6911                                1st round bba done   
6912  emittance measurements - 9491  ev (14.673  gev...

In [84]:
# Forgot to create dummy variables

# Dropping the first column rn is FACET so this is ok. May need to change later when there are multiple areas.
df = pd.concat([df,pd.get_dummies(df.tag, drop_first = True)], axis=1)
#df = df.drop(['FACET'], axis = 1)   #Taken care of by above line

In [85]:
df.head()

elogid   tag                                               text  \
0  137674  LCLS                                                      
1  137670  LCLS  A. Prinz approves this change. Approval grante...   
2  148112  LCLS  PARANOIA was restarted with changes to handle ...   
3  145825  LCLS  15-MAY-2007 20:24:11  %CAU-E-EPICS_MSG_PEP  CM...   
4  144311  LCLS  As one can see from Electrical Safety label th...   

                                     title  \
0          EPICS LCLS IOC heartbeat fault.   
1                       RR monitor to 2 Hz   
2                         PARANOIA Restart   
3               Errorlog re: LCLS SOLN 121   
4  Rack Location of LCLS BX01/BX02 Breaker   

                                      title_and_text  LCLS  
0                   epics lcls ioc heartbeat fault.      1  
1  rr monitor to 2 hz a. prinz approves this chan...     1  
2  paranoia restart paranoia was restarted with c...     1  
3  errorlog re: lcls soln 121 15-may-2007 20:24:1...     1  
4  rack location of lcls bx01/bx02 breaker as one...     1

In [87]:
X = df['title_and_text'] 
Y = df.drop(['text','title','elogid', 'title_and_text','LCLS'], axis = 1)
#Y = df.drop(['text','title','elogid', 'title_and_text'], axis = 1)

In [88]:
# Creat tokenizer function
def tokenize(x):
    
    # Generating list of stop words
    stop_words = set(stopwords.words('english'))
    
    # Separate sentance into individual words
    no_punctuation_x = re.sub(r"[^a-zA-Z0-9]"," ", x)
    word_token = word_tokenize(no_punctuation_x)
    
    # Lemmatizing each word and added cleaned words to clean_words
    lemmatizer = WordNetLemmatizer()
    clean_words = []
    for word in word_token:
        clean_words.append(lemmatizer.lemmatize(word.lower().strip()))

    # Return lematized words that are indeed words and are not in stopwords list
    final_token = [w for w in clean_words if w not in stop_words]
    return final_token

In [89]:
# Proof that it works. Watch out for cells with tables tho, there is a lot of useless information in them.
#tokenize(df.at[6912,'title_and_text'])

In [90]:
pipeline = Pipeline([
    ('vectorizer', CountVectorizer(tokenizer = tokenize)),
    ('tfidf', TfidfTransformer()),
    ('clf', DecisionTreeClassifier())
    #('clf', MultiOutputClassifier(RandomForestClassifier()))
    #('clf', MultiOutputClassifier(AdaBoostClassifier(base_estimator=DecisionTreeClassifier(max_depth=1, class_weight='balanced'))))
])

In [91]:
X_train, X_test, y_train, y_test = train_test_split(X,Y)
pipeline.fit(X_train, y_train)

Pipeline(memory=None,
     steps=[('vectorizer', CountVectorizer(analyzer='word', binary=False, decode_error='strict',
        dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
        lowercase=True, max_df=1.0, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words=None,
       ...      min_weight_fraction_leaf=0.0, presort=False, random_state=None,
            splitter='best'))])

In [92]:
y_pred = pipeline.predict(X_test)

In [93]:
# y_pred is not a data frame, let's make it into a data frame so we can see things more easily
df_ypred = pd.DataFrame(y_pred, columns = y_test.columns)
df_ypred.head()

tag
0  LCLS
1  LCLS
2  LCLS
3  LCLS
4  LCLS

In [94]:
# Will give "scoring" for each tag. Right now it's only one, but this allows us to check in case we have >2 programs
for col in y_test.columns:
    print(col)
    print(classification_report(y_test[col].values, df_ypred[col]))

tag
              precision    recall  f1-score   support

       FACET       0.78      0.77      0.78       319
        LCLS       0.95      0.95      0.95      1410

   micro avg       0.92      0.92      0.92      1729
   macro avg       0.87      0.86      0.86      1729
weighted avg       0.92      0.92      0.92      1729



In [95]:
pipeline.get_params()

{'memory': None,
 'steps': [('vectorizer',
   CountVectorizer(analyzer='word', binary=False, decode_error='strict',
           dtype=<class 'numpy.int64'>, encoding='utf-8', input='content',
           lowercase=True, max_df=1.0, max_features=None, min_df=1,
           ngram_range=(1, 1), preprocessor=None, stop_words=None,
           strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
           tokenizer=<function tokenize at 0x1a2e407158>, vocabulary=None)),
  ('tfidf',
   TfidfTransformer(norm='l2', smooth_idf=True, sublinear_tf=False, use_idf=True)),
  ('clf',
   DecisionTreeClassifier(class_weight=None, criterion='gini', max_depth=None,
               max_features=None, max_leaf_nodes=None,
               min_impurity_decrease=0.0, min_impurity_split=None,
               min_samples_leaf=1, min_samples_split=2,
               min_weight_fraction_leaf=0.0, presort=False, random_state=None,
               splitter='best'))],
 'vectorizer': CountVectorizer(analyzer='word', binary

In [96]:
example = pd.Series('klystron 16-4 acting up')
pipeline.predict(example)

array(['LCLS'], dtype=object)

In [78]:
df[df['text'].str.contains('FACET') == True]

elogid    tag                                               text  \
588   515398  FACET      * Recover beam to FACET dump & scav ext. l...   
617   515325  FACET  Released new version of IOCManager and booted ...   
667   515109  FACET  <b>FACET Delivery</b>: \n<u>delivered</u>: 7 h...   
685   515030  FACET  Due to the SB17 issue and the Sector 9 klystro...   
697   514946  FACET  Low current (3e9) beam in Linac to Sector 19 a...   
776   514425  FACET  BCON is 90 BDES was 12.\nHSTB indicates BDES l...   
777   514424  FACET  TGUN off BAS.\nA few magnets here and there a ...   
954   511715  FACET  <p><a href="https://slacspace.slac.stanford.ed...   
1078  509414  FACET  <b>General program</b>: \n\n1) FACET quad BBA\...   
1079  509415  FACET  Cater #95089.\n\nThis is important because the...   
1088  509402  FACET  State of the machine: 10Hz e- 3e9 TMIT to FACE...   
1112  509268  FACET  <b>Program</b>: Match LI02 following new steer...   
1142  509122  FACET  Delivered: 3 hrs e- to FACET; Down: 5 hrs (DR1...   
1150  509110  FACET  We Catered this last owl shift but needed beam...   
1155  509100  FACET  State of machine: 19.5 GeV (model) e-, 5e9 TMI...   
1198  508865  FACET  Fairly ambitious program for tonight:\n\n1. Re...   
1214  508701  FACET  <b>Summary</b>\n\nWe performed scans of final ...   
1218  508681  FACET  Looks like camera wasn't triggering for USOTR....   
1239  508499  FACET  I also added a "Print Ops Log" option.\nYou ca...   
1240  508447  FACET  <b>Program</b>\n\n1) Deliver stable beam at 1....   
1242  508431  FACET  FACET LI19 LIONS show a negative voltage when ...   
1244  508424  FACET                              1.5e10 to FACET Dump.   
1268  506122  FACET  Fisher was able to get the EPICs PLIC signal f...   
1301  505904  FACET  From FACET log:\n\nAEG found this 3/4 inch bea...   
1313  528354  FACET  Data saved as unformated matlab data for some ...   
1317  528348  FACET  Current state of machine: 9Hz e- to PR185, 1Hz...   
1318  528346  FACET  After stdzing the NDR mags with the access job...   
1401  527125  FACET  I have built and CVSed version 1 of an Ops Das...   
1412  526219  FACET  From FACET Home -> BPM-LI20, the BL31 button s...   
1487  525973  FACET  The path to get to these is: Main index -> MPS...   
...      ...    ...                                                ...   
3401  535443   LCLS  <p>&nbsp;</p><table border="0" cellpadding="0"...   
3404  535273  FACET                         Both columns, LCLS & FACET   
3437  534841   LCLS  <table border="0" cellpadding="0" cellspacing=...   
3586  533745  FACET  Epic FACET shift tonight!\n\n<b>Program</b>:\n...   
3601  533721  FACET  No luck with either the BNS for 2-6 & 11-19, o...   
3606  533707  FACET  Turner is on shift to help us work on the bunc...   
3627  533599  FACET  Seems to be a ~400MeV energy difference betwee...   
3649  533526  FACET  <b>Program</b>: (8 hrs MD delivered)\n\n1. Rec...   
3650  533530  FACET  9Hz to FACET dump, 1.5e10 Nel\n1600+ on pyro\n...   
3659  533503  FACET  In preparation for going to 21 GeV (per Uli an...   
3665  533493  FACET  <b>The original program</b> was to: phase SB 2...   
3707  533360  FACET  We believe this was due to OPS doing LINAC BBA...   
3747  531661  FACET  This corrector wouldn't go back after the R12 ...   
3748  531657  FACET  State of machine: 9Hz to FACET dump, 5e9 Nel T...   
3763  531601  FACET  <b>Program</b>:\n\n1. Dispersion studies with ...   
3799  531475  FACET  A lot of stuff, in LCLS and FACET seems to hav...   
3866  530345  FACET  Program\n\n    *  Restore "pencil" beam (5E9, ...   
3882  530309  FACET  Ops: Hollosi, Melton\nDelivered: 4 hrs, beam r...   
3885  530298  FACET  We finally got the ext. kicker timed in, the V...   
4076  528800  FACET  It appears that they skipped the QS quads for ...   
4081  528783  FACET  Looks like it had jumped a few times on Day ne...   
4084  528769  FACET  Program:\nFACET chicane BBA\nChasing the 3120 ...   
4089  5287

In [79]:
df.at[4142,'title_and_text']

'ep01 mps hlam enable fault with ep01 stoppers in we think this should be bypassed when we put in the positron vault stoppers-- the hlam disable is correctly bypassed and the magnet came back on.'